In [1]:
import numpy as np
from sklearn import datasets

In [2]:
#                    Description of the dataset
#  data : {ndarray, dataframe} of shape (150, 4)
#             The data matrix. If `as_frame=True`, `data` will be a pandas
#             DataFrame.
#         target: {ndarray, Series} of shape (150,)
#             The classification target. If `as_frame=True`, `target` will be
#             a pandas Series.
#         feature_names: list
#             The names of the dataset columns.
#         target_names: list
#             The names of target classes.
#         frame: DataFrame of shape (150, 5)
#             Only present when `as_frame=True`. DataFrame with `data` and
#             `target`.

In [3]:
# Creating an output array
def target_array(raw_target):
    target = []
    for i in range(len(raw_target)):
        if raw_target[i] == 0:
            target.append([1, 0, 0])
        elif raw_target[i] == 1:
            target.append([0, 1, 0])
        else:
            target.append([0, 0, 1])
    return (np.array(target, dtype="uint8"))
    

In [4]:
# Loading the Iris dataset
iris = datasets.load_iris()

data = iris.data
target = target_array(iris.target)

# Mixing data
np.random.seed(1)
np.random.shuffle(data)
np.random.seed(1)
np.random.shuffle(target)

In [5]:
# RELU
# # Activation function
# def relu(x): 
#     z = (x > 0) * x     # returns x if x > 0
#     return z            # return 0 otherwise


# # Derivative of the activation function
# def relu2deriv(output):
#     z = output > 0
#     return z            # returns 1 for input > 0


# Sigmoid 
# Activation function
def activation_func(x):
    z = 1 / (1 + (np.e ** -x))
    return z


# Derivative of the activation function
def deriv_activation_func(x):
    z = activation_func(x)
    return z * (1 - z)
    

0 - input layer  
1 - hidden layer  
2 - output layer  
weights_1_to_2 - Weights linking the hidden layer and the output layer

In [6]:
# Neural network learning function
def neural_network_training(weights_0_to_1,
                            weights_1_to_2,
                            data,
                            target,
                            number_training_data,
                            epochs=5,
                            alpha=0.5):
    epoch_print_step = 20
    for epoch in range(epochs):
        if epoch % epoch_print_step == 0:
            print(f"\nepoch: {epoch + 1}")
        for i in range(number_training_data):
            # Forward propagation
            layer_0 = data[i:i + 1]
            layer_1 = activation_func(np.dot(layer_0, weights_0_to_1))
            layer_2 = np.dot(layer_1, weights_1_to_2)

            # Error calculation
            layer_2_delta = layer_2 - target[i:i + 1]
            layer_1_delta = layer_2_delta.dot(weights_1_to_2.T) * deriv_activation_func(layer_1)
            
            # Backpropagation
            weights_1_to_2 -= alpha * layer_1.T.dot(layer_2_delta)
            weights_0_to_1 -= alpha * layer_0.T.dot(layer_1_delta)
            
            # Output of a part of the data
            if epoch % epoch_print_step == 0 and i >= 10 and i <= 13:
                print(f"{np.argmax(layer_2) == np.argmax(target[i])} Error: {np.sum((layer_2 - target[i]) ** 2):.5f}, out = {layer_2}, target = {target[i]}")
  
    return weights_0_to_1, weights_1_to_2

In [7]:
# Declaring parameters for a neural network
# epochs = 1000
epochs = 401
alpha = 0.002

number_test_data = 30
len_data = len(data)
number_training_data = len_data - number_test_data
layer_1_size = 6

np.random.seed(1)
weights_0_to_1 = np.random.rand(len(data[0]), layer_1_size)
weights_1_to_2 = np.random.rand(layer_1_size, len(target[0]))

In [8]:
# Neural network training
weights_0_to_1, weights_1_to_2 = neural_network_training(weights_0_to_1,
                                                         weights_1_to_2,
                                                         data,
                                                         target,
                                                         number_training_data,
                                                         epochs,
                                                         alpha)


epoch: 1
True Error: 18.96789, out = [[2.59977008 3.42636901 2.51432246]], target = [0 1 0]
False Error: 18.76972, out = [[2.48188846 3.27912205 2.41269308]], target = [1 0 0]
False Error: 20.90407, out = [[2.6055155  3.43673034 2.51797506]], target = [0 0 1]
True Error: 17.78250, out = [[2.52926479 3.32898957 2.4415429 ]], target = [0 1 0]

epoch: 21
False Error: 0.77223, out = [[0.27571537 0.28793455 0.43494565]], target = [0 1 0]
True Error: 0.55497, out = [[0.39437774 0.33728639 0.27280926]], target = [1 0 0]
True Error: 0.41687, out = [[0.23320531 0.26123642 0.45756403]], target = [0 0 1]
False Error: 0.77019, out = [[0.26006873 0.26436333 0.40173672]], target = [0 1 0]

epoch: 41
False Error: 0.77146, out = [[0.240676   0.28644533 0.45208381]], target = [0 1 0]
True Error: 0.36769, out = [[0.46555898 0.27530332 0.07918902]], target = [1 0 0]
True Error: 0.29797, out = [[0.18996904 0.3369863  0.61486859]], target = [0 0 1]
False Error: 0.74581, out = [[0.25013608 0.27364106 0.394

In [9]:
def сhecking_accuracy(data, target):    
    if np.argmax(data) == np.argmax(target):
        print("True")
        return
    print(f"False: {data}, {target}")

In [10]:
def neural_network_test(weights_0_to_1,
                        weights_1_to_2,
                        data,
                        target,
                        number_test_data,
                        len_data):
    # print(len_data - number_test_data, len_data)
    for i in range(len_data - number_test_data, len_data):
        layer_0 = data[i:i + 1]
        layer_1 = activation_func(np.dot(layer_0, weights_0_to_1))
        layer_2 = np.dot(layer_1, weights_1_to_2)
        # error = np.sum((layer_2 - target[i]) ** 2)
        
        сhecking_accuracy(layer_2, target[i])
        # print(f"target = out: {np.argmax(target[i]) == np.argmax(layer_2)}\terror = {error:.4f}")
        # print(f"target = {target[i]}, out = {layer_2},\terror = {error:.4f}")

In [11]:
neural_network_test(weights_0_to_1,
                    weights_1_to_2,
                    data,
                    target,
                    number_test_data,
                    len_data)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False: [[0.09925872 0.3790793  0.5091387 ]], [0 1 0]
True
True
True
True
True
True
True
True
True
True
False: [[0.07677834 0.37936422 0.54149024]], [0 1 0]
True
True


In [12]:
def neural_network(weights_0_to_1,
                   weights_1_to_2,
                   data):
    layer_0 = data
    layer_1 = activation_func(np.dot(layer_0, weights_0_to_1))
    layer_2 = np.dot(layer_1, weights_1_to_2)    
    return layer_2

In [13]:
for i in range(20):
    number = np.random.randint(0, 150)
    prediction = neural_network(weights_0_to_1,
                                weights_1_to_2,
                                data[number])
    сhecking_accuracy(prediction, target[number])

True
True
True
True
False: [0.12688226 0.40534757 0.46324734], [0 1 0]
True
True
True
True
True
True
True
False: [0.09925872 0.3790793  0.5091387 ], [0 1 0]
True
True
True
True
True
True
True
